<a href="https://colab.research.google.com/github/jackykwok2002/Resume_model_train/blob/main/resume_model_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd # load and manipulate data and for One-Hot Encoding
import numpy as np # calculate the mean and standard deviation
import xgboost as xgb # XGBoost stuff
from sklearn.model_selection import train_test_split # split  data into training and testing sets
from sklearn.metrics import balanced_accuracy_score, roc_auc_score, make_scorer # for scoring during cross validation
from sklearn.model_selection import GridSearchCV # cross validation
from sklearn.metrics import confusion_matrix # creates a confusion matrix
from sklearn.metrics import plot_confusion_matrix # draws a confusion matrix
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OrdinalEncoder

In [ ]:
df = pd.read_csv("sdfz_resume.csv")
df = df.drop(['原始文本','姓名','投递时间','电话','邮箱','岗位','工作经历','技能'], axis=1) #temporarily drop 工作经历
df = df.drop_duplicates() 

In [ ]:
# 工作年限转换为Integer
def toNumber(x):
  if x["工作年限"] is None:
    return 

  s = str(x["工作年限"])

  if '毕业' in s:
    return 0
  elif '1年及以内' in s or '1年以内' in s:
    return 0.5
  else:
    import re
    return re.sub("[^0-9]", "", s)

df["工作年限"] = df.apply(toNumber, axis=1)
df['工作年限'] = pd.to_numeric(df['工作年限'])
df['工作年限'].unique()

# 将学历过滤
def filterEducation(x):
  if '深圳' in str(x["学历"]):
    return None 
  else:
    return x["学历"]

df["学历"] = df.apply(filterEducation, axis=1)

# 性别过滤
def filterSex(x):
  if '男' in str(x["性别"]) or '女' in str(x["性别"]):
    return x["性别"]
  else:
    return None

df["性别"] = df.apply(filterSex, axis=1)


In [ ]:
# 毕业时间
def filterGraduation(x):
  s = str(x["毕业时间"])
  if '-' in s:
    s = s.split('-')[0]
  if '/' in s:
    s = s.split('/')[2]
  s = s.replace("届", "").replace('级别','').replace('级','').replace('15659420518','0').replace('nan','0')
  return s

df["毕业时间"] = df.apply(filterGraduation, axis=1)
df['毕业时间'] = pd.to_numeric(df['毕业时间'])

In [ ]:
# Use number to represent schools
# 学校转换为数字

global school_lst
school_lst = list(df['毕业院校'].unique().astype(str).copy())
school_lst.sort()

df['毕业院校'] = df['毕业院校'].apply(lambda x: school_lst.index(str(x)))


In [ ]:
# 处理“技能”
# 转换为list
# def skillsToList(x):
#   if str(x["技能"]) == 'nan':
#     return []
#   return str(x["技能"]).split(' | ')
# df["技能"] = df.apply(skillsToList, axis=1)

# # One Hot Encoded #source:https://stackoverflow.com/questions/45312377/how-to-one-hot-encode-from-a-pandas-column-containing-a-list
# from sklearn.preprocessing import MultiLabelBinarizer
# mlb = MultiLabelBinarizer()
# df = df.join(pd.DataFrame(mlb.fit_transform(df.pop('技能')),
#                           columns=mlb.classes_,
#                           index=df.index))

In [ ]:
# Split the Data into Dependent and Independent Variables
X = df.drop('应聘职位', axis=1).copy() 
y = df['应聘职位'].copy().astype(str)

In [ ]:
# One Hot Encode the rest
X_encoded = pd.get_dummies(X, columns=['专业', 
                                       '学历', 
                                       '工作地点', 
                                       '性别',
                                       '类别',
                                       '应聘渠道'])

In [ ]:
le = LabelEncoder()
y = le.fit_transform(y)

In [ ]:
# Spliting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size = 0.2, random_state = 6688)

In [ ]:
clf_xgb = xgb.XGBClassifier(objective='multi:softmax',
                            eval_metric="logloss", ## this avoids a warning...
                            seed=42, 
                            use_label_encoder=False)
clf_xgb.fit(X_train, 
            y_train,
            verbose=True,
            early_stopping_rounds=10,
            eval_metric='mlogloss',
            eval_set=[(X_test, y_test)])

[0]	validation_0-mlogloss:3.94116
Will train until validation_0-mlogloss hasn't improved in 10 rounds.
[1]	validation_0-mlogloss:3.65526
[2]	validation_0-mlogloss:3.46611
[3]	validation_0-mlogloss:3.32155
[4]	validation_0-mlogloss:3.20425
[5]	validation_0-mlogloss:3.10451
